In [ ]:
import pandas as pd
import boto3,  json

### Step 1 is complete.
created:

- Vector collection: bw-rag-2026
- OpenSearch endpoint: 0bqbkd1pbc6xjbx7zd6k.eu-central-1.aoss.amazonaws.com

In [2]:
sts = boto3.client("sts")
print(sts.get_caller_identity())


{'UserId': 'AROA4DL4NCWVIRXUCNB4F:SageMaker', 'Account': '831873947050', 'Arn': 'arn:aws:sts::831873947050:assumed-role/AmazonSageMaker-ExecutionRole-20260213T130476/SageMaker', 'ResponseMetadata': {'RequestId': 'f405116f-1c7f-4db8-b296-2d0dda0168d2', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'f405116f-1c7f-4db8-b296-2d0dda0168d2', 'x-amz-sts-extended-request-id': 'MTp1cy1lYXN0LTE6UzoxNzcxMzIxMzUyNzg5Okc6Y2J1UjRRbFc=', 'content-type': 'text/xml', 'content-length': '470', 'date': 'Tue, 17 Feb 2026 09:42:32 GMT'}, 'RetryAttempts': 0}}


## Step 2 — Load CSV from S3 and prepare text for embeddings

In [3]:
s3_path = "s3://bw-odata-export-eli-2026/Mappe1.csv"

In [4]:
df = pd.read_csv(s3_path, sep=";", dtype=str).fillna("")
df.head()


,T_Transformation_ID,T_owner,T_sourcetype,T_sourcesubtype,T_Source_Object,T_Targettype,T_Targetsubtype,T_Target_Object,T_Progr_ID_GEN,F_Segmentnummer,F_Regelnummer,F_Regelschrittes,F_Regelparameters,F_Feldname1,F_Feldname2,F_Strukturfeldes,F_Schluesselfeld,F_Position,F_Aggregationsverhalten
0,0DE3KLEVQQUL3YCP1FJRNDLOBZW530NL,DUSDALD,RSDS,,0BPARTNER_TEXT EADCLNT100,IOBJ,TEXT,ZBPARTNER,28S9JS0V8JZU5UDJN377OFJY2,0001,3,0,0,PARTNER,PARTNER,F,WAHR,0001,
1,0DE3KLEVQQUL3YCP1FJRNDLOBZW530NL,DUSDALD,RSDS,,0BPARTNER_TEXT EADCLNT100,IOBJ,TEXT,ZBPARTNER,28S9JS0V8JZU5UDJN377OFJY2,0001,3,0,1,ZBPARTNER,ZBPARTNER,I,WAHR,0001,
2,0DE3KLEVQQUL3YCP1FJRNDLOBZW530NL,DUSDALD,RSDS,,0BPARTNER_TEXT EADCLNT100,IOBJ,TEXT,ZBPARTNER,28S9JS0V8JZU5UDJN377OFJY2,0001,4,0,1,0LANGU,0LANGU,I,WAHR,0005,
3,0GHKEN2UR7L75K43WFTD5XWGYAAZD8SP,MARTINA,RSDS,,ZARMA_TRANSPONIEREN FILE,ADSO,,ZTAB_DAT,3MSQF6JFUJ0LT32Q6K1L3YH9G,0001,1,0,0,KUNDE,KUNDE,F,FALSCH,0001,
4,0GHKEN2UR7L75K43WFTD5XWGYAAZD8SP,MARTINA,RSDS,,ZARMA_TRANSPONIEREN FILE,ADSO,,ZTAB_DAT,3MSQF6JFUJ0LT32Q6K1L3YH9G,0001,1,0,1,KUNDE,KUNDE,F,WAHR,0004,


In [5]:
df.columns.tolist()

['T_Transformation_ID',
 'T_owner',
 'T_sourcetype',
 'T_sourcesubtype',
 'T_Source_Object',
 'T_Targettype',
 'T_Targetsubtype',
 'T_Target_Object',
 'T_Progr_ID_GEN',
 'F_Segmentnummer',
 'F_Regelnummer',
 'F_Regelschrittes',
 'F_Regelparameters',
 'F_Feldname1',
 'F_Feldname2',
 'F_Strukturfeldes',
 'F_Schluesselfeld',
 'F_Position',
 'F_Aggregationsverhalten']

In [6]:
df.head(1).T

,0
T_Transformation_ID,0DE3KLEVQQUL3YCP1FJRNDLOBZW530NL
T_owner,DUSDALD
T_sourcetype,RSDS
T_sourcesubtype,
T_Source_Object,0BPARTNER_TEXT EADCLNT100
T_Targettype,IOBJ
T_Targetsubtype,TEXT
T_Target_Object,ZBPARTNER
T_Progr_ID_GEN,28S9JS0V8JZU5UDJN377OFJY2
F_Segmentnummer,0001


In [7]:
!pip -q install opensearch-py requests-aws4auth


## Step 3- Turn each row into a “document text”

This is what the LLM will be grounded on. LLMs cannot read tables directly. They work with text. So we convert **table row → natural text sentence**

In [8]:
def row_to_text(r):
    return (
        f"Transformation_ID={r.get('T_Transformation_ID','')}; "
        f"Owner={r.get('T_owner','')}; "
        f"Source_Object={r.get('T_Source_Object','')}; "
        f"Target_Object={r.get('T_Target_Object','')}; "
        f"Field1={r.get('F_Feldname1','')}; "
        f"Field2={r.get('F_Feldname2','')}; "
        f"RuleParams={r.get('F_Regelparameters','')}; "
        f"Agg={r.get('F_Aggregationsverhalten','')}"
    )

df["doc_text"] = df.apply(row_to_text, axis=1)

df[["doc_text"]].head()


,doc_text
0,Transformation_ID=0DE3KLEVQQUL3YCP1FJRNDLOBZW5...
1,Transformation_ID=0DE3KLEVQQUL3YCP1FJRNDLOBZW5...
2,Transformation_ID=0DE3KLEVQQUL3YCP1FJRNDLOBZW5...
3,Transformation_ID=0GHKEN2UR7L75K43WFTD5XWGYAAZ...
4,Transformation_ID=0GHKEN2UR7L75K43WFTD5XWGYAAZ...


#### This text will be:

- embedded

- stored in OpenSearch

- used to answer questions

## Step 4 - Create embeddings with Bedrock (Titan)

Titan embeddings are used for semantic search.

- An embedding converts each doc_text into a numeric vector.
- OpenSearch will store these vectors and later retrieve the most relevant rows for a question.

In [9]:
bedrock = boto3.client("bedrock", region_name="eu-central-1")

resp = bedrock.list_foundation_models()
embedding_models = [
    m for m in resp["modelSummaries"]
    if "EMBEDDING" in m.get("modelCapabilities", []) or m.get("outputModalities") == ["EMBEDDING"]
]

for m in embedding_models:
    print(m["modelId"], "|", m.get("modelName"))

cohere.embed-v4:0 | Embed v4
amazon.titan-embed-text-v1:2:8k | Titan Embeddings G1 - Text
amazon.titan-embed-text-v1 | Titan Embeddings G1 - Text
amazon.titan-embed-image-v1:0 | Titan Multimodal Embeddings G1
amazon.titan-embed-image-v1 | Titan Multimodal Embeddings G1
amazon.titan-embed-text-v2:0 | Titan Embeddings G2 - Text
cohere.embed-english-v3 | Embed English
cohere.embed-multilingual-v3 | Embed Multilingual


In [10]:
br = boto3.client("bedrock-runtime", region_name="eu-central-1")

EMBED_MODEL_ID = "amazon.titan-embed-text-v2:0"  # if this errors, we will adjust

def embed_one(text: str):
    body = json.dumps({"inputText": text})
    resp = br.invoke_model(
        modelId=EMBED_MODEL_ID,
        body=body,
        accept="application/json",
        contentType="application/json",
    )
    out = json.loads(resp["body"].read())
    return out["embedding"]

vec = embed_one(df["doc_text"].iloc[0])

print("dimension:", len(vec))
print("sample:", vec[:5])

dimension: 1024
sample: [-0.002984684659168124, 0.022916307672858238, 0.08112318068742752, 0.008177528157830238, 0.061119381338357925]


## Step 5 — Create the OpenSearch index (vector) and load a first batch

What this step does

- Connect SageMaker → OpenSearch Serverless endpoint

- Create an index with a knn_vector field of dimension 1024

- Index (store) the first N rows with their embeddings

- endpoint:
0bqbkd1pbc6xjbx7zd6k.eu-central-1.aoss.amazonaws.com

### 5.1 Install OpenSearch client libs

In [11]:
!pip -q install opensearch-py requests-aws4auth


### 5.2 Connect to OpenSearch Serverless

In [12]:
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
import requests

In [13]:
!pip -q install requests requests-aws4auth

In [14]:
sts = boto3.client("sts")
print(sts.get_caller_identity())


{'UserId': 'AROA4DL4NCWVIRXUCNB4F:SageMaker', 'Account': '831873947050', 'Arn': 'arn:aws:sts::831873947050:assumed-role/AmazonSageMaker-ExecutionRole-20260213T130476/SageMaker', 'ResponseMetadata': {'RequestId': '59e669a1-c983-4a40-b790-b766cf93d425', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '59e669a1-c983-4a40-b790-b766cf93d425', 'x-amz-sts-extended-request-id': 'MTp1cy1lYXN0LTE6UzoxNzcxMzIxMzY1Nzk1Okc6R1RQU3pQUjA=', 'content-type': 'text/xml', 'content-length': '470', 'date': 'Tue, 17 Feb 2026 09:42:45 GMT'}, 'RetryAttempts': 0}}


In [15]:
region = "eu-central-1"
service = "aoss"

session = boto3.Session()
creds = session.get_credentials().get_frozen_credentials()

awsauth = AWS4Auth(
    creds.access_key,
    creds.secret_key,
    region,
    service,
    session_token=creds.token
)

url = "https://0bqbkd1pbc6xjbx7zd6k.eu-central-1.aoss.amazonaws.com/connectivity-test"

r = requests.put(
    url,
    auth=awsauth,
    headers={"Content-Type": "application/json"},
    json={
        "settings": {"index": {"knn": True}},
        "mappings": {"properties": {"vector": {"type": "knn_vector", "dimension": 1024}}}
    }
)

print("status:", r.status_code)
print("body:", r.text[:1200])


status: 400
body: {"error":{"root_cause":[{"type":"resource_already_exists_exception","reason":"OpenSearch exception [type=resource_already_exists_exception, reason=index [connectivity-test/YSj7apwBoclqXtdwGpWj] already exists]- server : [envoy]"}],"type":"resource_already_exists_exception","reason":"OpenSearch exception [type=resource_already_exists_exception, reason=index [connectivity-test/YSj7apwBoclqXtdwGpWj] already exists]- server : [envoy]"},"status":400}


In [16]:

print(boto3.client("sts").get_caller_identity()["Arn"])


arn:aws:sts::831873947050:assumed-role/AmazonSageMaker-ExecutionRole-20260213T130476/SageMaker


### 5.3 create the real vector index for your RAG data 

In [17]:
region = "eu-central-1"
service = "aoss"
endpoint = "https://0bqbkd1pbc6xjbx7zd6k.eu-central-1.aoss.amazonaws.com"
index_name = "bw-mappings"

session = boto3.Session()
creds = session.get_credentials().get_frozen_credentials()
awsauth = AWS4Auth(creds.access_key, creds.secret_key, region, service, session_token=creds.token)

index_body = {
    "settings": {
        "index": {
            "knn": True
        }
    },
    "mappings": {
        "properties": {
            "vector": {"type": "knn_vector", "dimension": 1024},
            "doc_text": {"type": "text"},
            "source": {"type": "keyword"}  # optional
        }
    }
}

r = requests.put(
    f"{endpoint}/{index_name}",
    auth=awsauth,
    headers={"Content-Type": "application/json"},
    data=json.dumps(index_body)
)

print("status:", r.status_code)
print("body:", r.text[:800])


status: 400
body: {"error":{"root_cause":[{"type":"resource_already_exists_exception","reason":"OpenSearch exception [type=resource_already_exists_exception, reason=index [bw-mappings/1d37apwB7utvBO6nJude] already exists]- server : [envoy]"}],"type":"resource_already_exists_exception","reason":"OpenSearch exception [type=resource_already_exists_exception, reason=index [bw-mappings/1d37apwB7utvBO6nJude] already exists]- server : [envoy]"},"status":400}


### 5.4 — Verify mapping (confirm index exists + 1024)

In [18]:
r = requests.get(f"{endpoint}/{index_name}", auth=awsauth)
print("status:", r.status_code)
print("body:", r.text[:800])


status: 200
body: {"bw-mappings":{"aliases":{},"mappings":{"properties":{"doc_text":{"type":"text"},"metadata":{"properties":{"T_Source_Object":{"type":"text","fields":{"keyword":{"type":"keyword","ignore_above":256}}},"T_Target_Object":{"type":"text","fields":{"keyword":{"type":"keyword","ignore_above":256}}},"T_Transformation_ID":{"type":"text","fields":{"keyword":{"type":"keyword","ignore_above":256}}},"T_owner":{"type":"text","fields":{"keyword":{"type":"keyword","ignore_above":256}}}}},"source":{"type":"keyword"},"vector":{"type":"knn_vector","dimension":1024}}},"settings":{"index":{"number_of_shards":"2","provided_name":"bw-mappings","knn":"true","creation_date":"1771243786009","number_of_replicas":"0","uuid":"Ud1bZpwB7utvBO6nTuD8","version":{"created":"136387827"}}}}}


## Step 6 — Load the first embedded documents into OpenSearch

- Defines an endpoint + index name
- Creates AWS SigV4 auth for OpenSearch Serverless
- Creates Bedrock runtime client for embeddings


In [43]:
region = "eu-central-1"
service = "aoss"
endpoint = "https://0bqbkd1pbc6xjbx7zd6k.eu-central-1.aoss.amazonaws.com"
index_name = "bw-mappings"

# IMPORTANT: set your real embedding model id here
EMBED_MODEL_ID = "amazon.titan-embed-text-v2:0"
#EMBED_MODEL_ID = "qwen.qwen3-235b-a22b-2507-v1:0"
#EMBED_MODEL_ID = "amazon.titan-embed-text-v1"

session = boto3.Session()
creds = session.get_credentials().get_frozen_credentials()

awsauth = AWS4Auth(
    creds.access_key,
    creds.secret_key,
    region,
    service,
    session_token=creds.token
)

br = boto3.client("bedrock-runtime", region_name=region)

### 6.2 Embedding function (text -> vector)

In [44]:
def embed_one(text: str) -> list[float]:
    body = json.dumps({"inputText": text})

    resp = br.invoke_model(
        modelId=EMBED_MODEL_ID,
        body=body,
        accept="application/json",
        contentType="application/json",
    )

    out = json.loads(resp["body"].read())
    return out["embedding"]


In [45]:
vec = embed_one(df["doc_text"].iloc[0])
print("dimension:", len(vec))
print("sample:", vec[:5])

dimension: 1024
sample: [-0.002984684659168124, 0.022916307672858238, 0.08112318068742752, 0.008177528157830238, 0.061119381338357925]


### 6.3 Bulk upload helper (_bulk API)
Builds an NDJSON payload for OpenSearch _bulk and uploads documents.
Each document includes:
- doc_text (text)
- vector (embedding)
- metadata (optional, useful later in answers)

In [47]:
def bulk_upload(docs: list[dict]):
    lines = []
    for d in docs:
        lines.append(json.dumps({"index": {"_index": index_name}}))
        lines.append(json.dumps(d))
    payload = "\n".join(lines) + "\n"

    r = requests.post(
        f"{endpoint}/_bulk",
        auth=awsauth,
        headers={"Content-Type": "application/x-ndjson"},
        data=payload
    )
    return r


### 6.4 Ingest a small batch (first 5 rows)
What this does
- Takes the first 5 rows from df
- Creates embeddings
- Uploads them to OpenSearch 

In [48]:
test_n = 5
docs = []

for i in range(min(test_n, len(df))):
    row = df.iloc[i]
    text = str(row["doc_text"])
    vec = embed_one(text)

    docs.append({
        "doc_text": text,
        "vector": vec,
        "metadata": {
            "T_Transformation_ID": row.get("T_Transformation_ID", ""),
            "T_Source_Object": row.get("T_Source_Object", ""),
            "T_Target_Object": row.get("T_Target_Object", ""),
            "T_owner": row.get("T_owner", "")
        }
    })

resp = bulk_upload(docs)
print("status:", resp.status_code)
print(resp.text[:1000])


status: 200
{"took":228,"errors":false,"items":[{"index":{"_index":"bw-mappings","_id":"1%3A0%3AumMma5wBFJgrhPAxT8x7","_version":1,"result":"created","_shards":{"total":0,"successful":0,"failed":0},"_seq_no":0,"_primary_term":0,"status":201}},{"index":{"_index":"bw-mappings","_id":"1%3A0%3Au2Mma5wBFJgrhPAxT8x7","_version":1,"result":"created","_shards":{"total":0,"successful":0,"failed":0},"_seq_no":0,"_primary_term":0,"status":201}},{"index":{"_index":"bw-mappings","_id":"1%3A0%3AvGMma5wBFJgrhPAxT8x7","_version":1,"result":"created","_shards":{"total":0,"successful":0,"failed":0},"_seq_no":0,"_primary_term":0,"status":201}},{"index":{"_index":"bw-mappings","_id":"1%3A0%3AvWMma5wBFJgrhPAxT8x7","_version":1,"result":"created","_shards":{"total":0,"successful":0,"failed":0},"_seq_no":0,"_primary_term":0,"status":201}},{"index":{"_index":"bw-mappings","_id":"1%3A0%3AvmMma5wBFJgrhPAxT8x7","_version":1,"result":"created","_shards":{"total":0,"successful":0,"failed":0},"_seq_no":0,"_primary_

### 6.5  Verify that documents are now in the index
Checks document count by retrieving 3 documents.

In [49]:
r = requests.get(
    f"{endpoint}/{index_name}/_search",
    auth=awsauth,
    headers={"Content-Type": "application/json"},
    data=json.dumps({"size": 3, "query": {"match_all": {}}})
)
print("status:", r.status_code)
print(r.text[:1200])


status: 200
{"took":38,"timed_out":false,"_shards":{"total":0,"successful":0,"skipped":0,"failed":0},"hits":{"total":{"value":20,"relation":"eq"},"max_score":1.0,"hits":[{"_index":"bw-mappings","_id":"1%3A0%3ApmN0ZpwBFJgrhPAxDsy5","_score":1.0,"_source":{"doc_text": "Transformation_ID=0DE3KLEVQQUL3YCP1FJRNDLOBZW530NL; Owner=DUSDALD; Source_Object=0BPARTNER_TEXT                EADCLNT100; Target_Object=ZBPARTNER; Field1=PARTNER; Field2=PARTNER; RuleParams=0; Agg=", "vector": [-0.002984684659168124, 0.022916307672858238, 0.08112318068742752, 0.008177528157830238, 0.061119381338357925, -0.003720151958987117, 0.05419207364320755, -0.029360590502619743, 0.019125109538435936, -0.0336088091135025, -0.06454852223396301, -0.004185734316706657, 0.07239019870758057, 0.015048522502183914, 0.02715606614947319, 0.1005830466747284, 0.02506488747894764, -0.03650009259581566, 0.027805248275399208, 0.03864268586039543, 0.025415483862161636, 0.09604794532060623, 0.01917516067624092, 0.003537504468113184,

## Step 7 — Vector search (question → embedding → kNN retrieval)
### 7.1 — kNN search function

This embeds the question, searches OpenSearch by vector similarity, and returns the top hits.

In [50]:
def knn_search_unique(question: str, k: int = 5, fetch_k: int = 20):
    q_vec = embed_one(question)

    query = {
        "size": fetch_k,  # fetch more, then dedupe
        "_source": ["doc_text", "metadata"],
        "query": {
            "knn": {
                "vector": {
                    "vector": q_vec,
                    "k": fetch_k
                }
            }
        }
    }

    r = requests.get(
        f"{endpoint}/{index_name}/_search",
        auth=awsauth,
        headers={"Content-Type": "application/json"},
        data=json.dumps(query),
    )
    r.raise_for_status()
    hits = r.json().get("hits", {}).get("hits", [])

    seen = set()
    unique = []
    for h in hits:
        t = h["_source"].get("doc_text", "")
        if t not in seen:
            seen.add(t)
            unique.append(h)
        if len(unique) >= k:
            break

    return unique


In [51]:
question = "Which transformation maps PARTNER to ZBPARTNER and what fields are involved?"
hits = knn_search_unique(question, k=5)

for i, h in enumerate(hits, 1):
    print("\n--- HIT", i, "score=", h.get("_score"))
    print(h["_source"]["doc_text"])


--- HIT 1 score= 0.5650803
Transformation_ID=0DE3KLEVQQUL3YCP1FJRNDLOBZW530NL; Owner=DUSDALD; Source_Object=0BPARTNER_TEXT                EADCLNT100; Target_Object=ZBPARTNER; Field1=0LANGU; Field2=0LANGU; RuleParams=1; Agg=

--- HIT 2 score= 0.5463005
Transformation_ID=0DE3KLEVQQUL3YCP1FJRNDLOBZW530NL; Owner=DUSDALD; Source_Object=0BPARTNER_TEXT                EADCLNT100; Target_Object=ZBPARTNER; Field1=PARTNER; Field2=PARTNER; RuleParams=0; Agg=

--- HIT 3 score= 0.51352954
Transformation_ID=0DE3KLEVQQUL3YCP1FJRNDLOBZW530NL; Owner=DUSDALD; Source_Object=0BPARTNER_TEXT                EADCLNT100; Target_Object=ZBPARTNER; Field1=ZBPARTNER; Field2=ZBPARTNER; RuleParams=1; Agg=

--- HIT 4 score= 0.464611
Transformation_ID=0GHKEN2UR7L75K43WFTD5XWGYAAZD8SP; Owner=MARTINA; Source_Object=ZARMA_TRANSPONIEREN           FILE; Target_Object=ZTAB_DAT; Field1=KUNDE; Field2=KUNDE; RuleParams=1; Agg=


## Step 8 — Bedrock answer (bilingual, only-from-data)

### 8.1 — Build bilingual answer prompt from retrieved hits

In [52]:
brc = boto3.client("bedrock", region_name=region)
models = brc.list_foundation_models(byOutputModality="TEXT")
for m in models["modelSummaries"][:30]:
    print(m["modelId"])


anthropic.claude-sonnet-4-20250514-v1:0
amazon.nova-2-lite-v1:0
anthropic.claude-haiku-4-5-20251001-v1:0
qwen.qwen3-235b-a22b-2507-v1:0
openai.gpt-oss-120b-1:0
twelvelabs.pegasus-1-2-v1:0
minimax.minimax-m2.1
anthropic.claude-sonnet-4-5-20250929-v1:0
anthropic.claude-opus-4-5-20251101-v1:0
qwen.qwen3-coder-30b-a3b-v1:0
qwen.qwen3-32b-v1:0
anthropic.claude-opus-4-6-v1
zai.glm-4.7-flash
openai.gpt-oss-20b-1:0
amazon.nova-pro-v1:0
amazon.rerank-v1:0
amazon.nova-lite-v1:0
amazon.nova-micro-v1:0
anthropic.claude-v2:1:18k
anthropic.claude-v2:1:200k
anthropic.claude-3-sonnet-20240229-v1:0
anthropic.claude-3-haiku-20240307-v1:0
anthropic.claude-3-5-sonnet-20240620-v1:0
anthropic.claude-3-7-sonnet-20250219-v1:0
cohere.rerank-v3-5:0
mistral.pixtral-large-2502-v1:0
meta.llama3-2-1b-instruct-v1:0
meta.llama3-2-3b-instruct-v1:0


In [54]:
chat = boto3.client("bedrock-runtime", region_name=region)
CHAT_MODEL_ID = "qwen.qwen3-235b-a22b-2507-v1:0"  # ON_DEMAND

def answer_bilingual_only_from_data(question: str, hits: list[dict]) -> str:
    context = "\n".join([h["_source"]["doc_text"] for h in hits])

    system_rules = (
        "You must answer ONLY using the provided CONTEXT.\n"
        "If the answer is not in the context, output exactly:\n"
        "DE: Nicht in den Daten.\n"
        "EN: Not in the data.\n"
        "Return exactly two sections in this format:\n"
        "DE:\n"
        "EN:\n"
    )

    resp = chat.converse(
        modelId=CHAT_MODEL_ID,
        messages=[
            {
                "role": "user",
                "content": [
                    {"text": f"{system_rules}\n\nCONTEXT:\n{context}\n\nQUESTION:\n{question}"}
                ],
            }
        ],
        inferenceConfig={"maxTokens": 500, "temperature": 0.0},
    )

    return resp["output"]["message"]["content"][0]["text"]



In [55]:
question = "Which transformation maps PARTNER to ZBPARTNER and what fields are involved?"

hits = knn_search_unique(question, k=5)

answer = answer_bilingual_only_from_data(question, hits)

print(answer)


DE:
Die Transformation mit der ID 0DE3KLEVQQUL3YCP1FJRNDLOBZW530NL bildet PARTNER auf ZBPARTNER ab. Die beteiligten Felder sind PARTNER und ZBPARTNER.
EN:
The transformation with ID 0DE3KLEVQQUL3YCP1FJRNDLOBZW530NL maps PARTNER to ZBPARTNER. The fields involved are PARTNER and ZBPARTNER.
